In [2]:
import json
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from pandasql import sqldf
from termcolor import colored

In [3]:
print(colored("""
Examples of queries to the Energy API 1:
Q1:
    select : *
Q2:
    select : *
    where  : region='Normandie'
Q3:
    select : region,date,consommation_brute_electricite_rte
    where  : region='Nouvelle-Aquitaine' and date='2022-03-27'
\n""", 'green'))

print(colored("""
Examples of queries to the Energy API 2:
Q1:
    select : *
Q2:
    select : *
    where  : region='Bretagne'
Q3:
    select : code, region, energie_injectee
    where  : code=44 and horodate='2019-12-31T23:00:00+00:00'
\n""", 'green'))

print(colored("""
Possibles columns in df2: [code_insee_region, date_heure, date, heure, region, consommation_brute_electricite_rte,consommation_brute_totale,consommation_brute_gaz_totale]"""
    , 'blue'))
print(colored("""Possibles columns in df2: [horodate, code, region, energie_injectee,nb_points_injection]"""
    , 'blue'))


Examples of queries to the Energy API 1:
Q1:
    select : *
Q2:
    select : *
    where  : region='Normandie'
Q3:
    select : region,date,consommation_brute_electricite_rte
    where  : region='Nouvelle-Aquitaine' and date='2022-03-27'



Examples of queries to the Energy API 2:
Q1:
    select : *
Q2:
    select : *
    where  : region='Bretagne'
Q3:
    select : code, region, energie_injectee
    where  : code=44 and horodate='2019-12-31T23:00:00+00:00'



Possibles columns in df2: [code_insee_region, date_heure, date, heure, region, consommation_brute_electricite_rte,consommation_brute_totale,consommation_brute_gaz_totale]
Possibles columns in df2: [horodate, code, region, energie_injectee,nb_points_injection]


In [5]:
def get_data_from_energy(list_of_tables,url):
    lista=list_of_tables
    response= rq.get(url)
    Repense = response.content
    parse_json = json.loads(Repense)
    n=parse_json['records'].__len__()
    list_data=[x for x in parse_json['records']]
    list_dic,df={},{}
    for i in range(n):
        list_dic[i]={key: list_data[i]["fields"][key] for key in lista}
    data=pd.DataFrame.from_dict(list_dic[0], orient='index').T
    for i in range(1,n):
        df[i]=pd.DataFrame.from_dict(list_dic[i], orient='index').T
        data=pd.concat([data,df[i]], axis=0)
    return data

In [6]:
list_api_1=['code_insee_region','date_heure', 'date', 'heure',  'region', 'consommation_brute_electricite_rte', 'consommation_brute_totale', 'consommation_brute_gaz_totale']
url1='https://odre.opendatasoft.com/api/records/1.0/search/?dataset=consommation-quotidienne-brute-regionale&q=&sort=-consommation_brute_gaz_grtgaz&facet=date_heure&facet=code_insee_region&facet=region'

list_api_2=['horodate','code','region','energie_injectee', 'nb_points_injection' ]
url2="""https://opendata.agenceore.fr/api/records/1.0/search/?dataset=production-demi-horaire-agregee-par-region&q=&sort=-horodate&facet=horodate&facet=region&facet=grd""" 

df1=get_data_from_energy(list_api_1,url1)
df1.head(3)

,code_insee_region,date_heure,date,heure,region,consommation_brute_electricite_rte,consommation_brute_totale,consommation_brute_gaz_totale
0,28,2022-03-27T01:00:00+00:00,2022-03-27,02:00,Normandie,1811,1811,0
0,75,2022-03-27T01:00:00+00:00,2022-03-27,02:00,Nouvelle-Aquitaine,4963,5052,89
0,75,2021-03-27T01:00:00+00:00,2021-03-27,02:00,Nouvelle-Aquitaine,4836,6455,1619


In [7]:
df2=get_data_from_energy(list_api_2,url2)
df2.head(3)

,horodate,code,region,energie_injectee,nb_points_injection
0,2019-12-31T23:00:00+00:00,28,Normandie,132332768,16041
0,2019-12-31T23:00:00+00:00,11,Île-de-France,207136975,16492
0,2019-12-31T23:00:00+00:00,44,Grand-Est,27279986,1181


Imagine we have this Globale query and we want to send each column to his corresponding api this we have to extract the column name that exist in each API so that is what we are goining to see in the following cells.

In [12]:
Query=""" 
SELECT df1.region,df2.region, df1.consommation_brute_electricite_rte, df2.energie_injectee
FROM df1
INNER JOIN df2 ON df1.region = df2.region
WHERE df2.region='Pays de la Loire'"""#print(Repense)
print(Query)
sqldf(Query)

 
SELECT df1.region,df2.region, df1.consommation_brute_electricite_rte, df2.energie_injectee
FROM df1
INNER JOIN df2 ON df1.region = df2.region
WHERE df2.region='Pays de la Loire'


,region,region,consommation_brute_electricite_rte,energie_injectee
0,Pays de la Loire,Pays de la Loire,2826,173684938
1,Pays de la Loire,Pays de la Loire,3318,173684938
2,Pays de la Loire,Pays de la Loire,3099,173684938


In [13]:
start = 'SELECT'#print tables_in_query
end = 'FROM'
tables_in_query=Query[Query.find(start)+len(start):Query.rfind(end)].strip().split(',')
print(tables_in_query)
list_api1 = []
list_api2 = []
list_api3 = []
for word in tables_in_query:
    if word.startswith("df1") or word.startswith(" df1"):#print tables_in_query
        list_api1.append(word.split('.')[1])
    elif word.startswith("df2") or word.startswith(" df2"):
        list_api2.append(word.split('.')[1])
    else:
        list_api3.append(word.split('.')[1])

['df1.region', 'df2.region', ' df1.consommation_brute_electricite_rte', ' df2.energie_injectee']


In [14]:
df1=get_data_from_energy(list_api1,url1)
df2=get_data_from_energy(list_api2,url2)

In [15]:
def get_response(df_1=df1,df_2=df2,df_3=df2,region='Pays de la Loire'):
#     join, left, right, inner, outer
#     union, concatenate ...
    matching_cols='region'
#     Final_Data=df_3.join((df_2.join(df_1.set_index(matching_cols),
#                        on=matching_cols)).set_index(matching_cols),
#                        on=matching_cols)
    Final_Data=(df_2.join(df_1.set_index(matching_cols), on=matching_cols))

    return Final_Data[Final_Data['region']==region]
df=get_response(df1,df2)
df

,region,energie_injectee,consommation_brute_electricite_rte
0,Pays de la Loire,173684938,2826
0,Pays de la Loire,173684938,3318
0,Pays de la Loire,173684938,3099


In [ ]:
# start = 'FROM'
# end = 'INNER'
# db_in_query=Query[Query.find(start)+len(start):Query.rfind(end)].strip().split(',')FROM df1, df2
# db_in_query

# start = 'INNER JOIN'
# end = 'WHERE'
# db_in_query=Query[Query.find(start)+len(start):Query.rfind(end)].strip().split(',')
# db_in_query